
# Генераторы и асинхронность.



Что такое генератор? Не каждый хороший питонист сможет правильно ответить на этот вопрос. Под капотом эта конструкция скрывает себе множество неочевидных вещей, в которых мы попробуем разобраться



In [ ]:

# В первую очередь генератор - это функция.
# Вместо ключевого слова `return` написан `yield`
# Любая функция с `yield` становится генератором

def gen(s: str):
    for x in s:
        yield x


In [ ]:

# Попробуем использовать наш генератор
gen('hello!')


In [ ]:
# Чёт ничего не получилось :( 
# На самом деле, при вызове, генератор не начинает выполнение функции,
# а создаёт объект `generator object`

g = gen('hello!')

for x in g:
    print(x)


In [ ]:
# На самом деле эта конструкция представима как

g = gen('hello!')

print(next(g))
print(next(g))
print(next(g))
print(next(g))
print(next(g))
print(next(g))

# Эксперимент! Вызовем next ещё раз
next(g)


In [ ]:
# Отлично, теперь мы знаем, как выполнять генератор до конца


g = gen('hello!')

while True:
    try:
        print(next(g))
    except StopIteration:
        break

Резумируем несколько важных вещей, про которые вы, возможно, уже догадались

 - Генератор - это функция
 - Генератор - это необычная функция
 - Генератор - это функция, которая умеет останавливаться!
 

In [ ]:

# В генераторе может быть несколько инструкций yield

def multigen(x: int):
    x += 1
    yield x
    
    x *= 42
    yield x
    
    x -= 10
    yield x


g = multigen(1)
print(next(g))

# Заметьте, что тут мы можем написать какой-то другой код
print('Here!')

print(next(g))
print(next(g))


In [ ]:


# Конструкция, которая ломает 80% питонистов.
# Оказывается, yield не только отдаёт данные, но и принимает их!

def bidirectional_gen():
    message = yield
    print(message)
    message2 = yield message.lower()
    print(message2)
    yield


g = bidirectional_gen()
g.send(None)  # равносильно next(g)

resp = g.send('HELLO!')
print(f"Got response from gen: {resp}")
g.send('World')


In [ ]:

def average():
    # Попробуем реализовать такую штуку
    pass


g = average()
g.send(None)
print(g.send(10))
print(g.send(10))
print(g.send(8))
print(g.send(8))


In [ ]:

def subgen(s: str):
    for x in s:
        print(x)

        
def delegator(s: str):
    




Поговорим про парадигмы асинхронного программирования <br>
https://ru.wikipedia.org/wiki/Round-robin_(%D0%B0%D0%BB%D0%B3%D0%BE%D1%80%D0%B8%D1%82%D0%BC)



### Пишем свою асинхронность!



In [ ]:
import time
import itertools
from inspect import getgeneratorstate


def first_task():
    for i in itertools.count():
        time.sleep(1)  # какие-то вычисления. На самом деле, всё работает совсем по другому.
        print(f'Iteration #{i}')
        yield


def second_task():
    i = 0
    while True:  
        time.sleep(1)
        i = (i + 1) % 5  # каждые 5 секунд
  
        if i == 0:
            print(time.time())
        yield
    

tasks = [first_task(), second_task()]


def run_event_loop(t):
    # FIXME!
        

run_event_loop(tasks)


In [ ]:
import asyncio


async def first_task():
    for i in itertools.count():
        await asyncio.sleep(1)  # настоящий асинх
        print(f'Iteration #{i}')
        yield


async def second_task():
    i = 0
    while True:  
        await asyncio.sleep(1)  # настоящий асинх
        i = (i + 1) % 5  # каждые 5 секунд
  
        if i == 0:
            print(time.time())
    

### TBA JIT

https://habr.com/ru/post/337420/